# 文本分类

In [1]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

In [3]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    gpu0 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

## 一个简单的例子

第一步是准备数据。 这里我们以 IMDB 数据集为例。

In [5]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

# set path to dataset
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")

classes = ["pos", "neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories=classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories=classes
)

84131840/84125825 [==============================] - 257s 3us/step


In [8]:
train_data.target

array([1, 0, 1, ..., 0, 0, 0])

In [9]:
train_data.target_names

['neg', 'pos']

In [15]:
len(train_data.data)

25000

In [16]:
x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)  # (25000,)
print(y_train.shape)  # (25000, 1)
print(x_train[0][:50])  # this film was just brilliant casting

(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


In [20]:
y_train

array([1, 0, 1, ..., 0, 0, 0])

第二步是运行TextClassifier。 作为一个快速演示，我们将 epochs 设置为 2。您还可以为自适应数量的 epochs 保留未指定的 epochs。

In [21]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=1
)  # It only tries 1 model as a quick demo.
# Feed the text classifier with training data.
clf.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 01m 03s]
val_loss: 0.2718275785446167

Best val_loss So Far: 0.2718275785446167
Total elapsed time: 00h 01m 03s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
782/782 [==============================] - 27s 33ms/step - loss: 0.4346 - accuracy: 0.7758
Epoch 2/2
782/782 [==============================] - 26s 33ms/step - loss: 0.2351 - accuracy: 0.9055
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


In [24]:
# Predict with the best model.
predicted_y = clf.predict(x_test)
predicted_y

782/782 [==============================] - 16s 20ms/step


array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [25]:
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

782/782 [==============================] - 20s 26ms/step - loss: 0.2654 - accuracy: 0.8938
[0.26544809341430664, 0.8938000202178955]


## 验证数据

默认情况下，AutoKeras 使用最后 20% 的训练数据作为验证数据。 如下例所示，您可以使用validation_split 来指定百分比。

In [26]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
)

您还可以使用自己的验证集，而不是使用`validation_data`将其从训练数据中分离出来。

In [27]:
split = 5000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    epochs=2,
    # Use your own validation set.
    validation_data=(x_val, y_val),
)

## 自定义搜索空间

对于高级用户，您可以使用 AutoModel 而不是 TextClassifier 自定义搜索空间。 您可以为一些高级配置配置 TextBlock，例如，针对要使用的文本矢量化方法类型的矢量化器。 您可以使用“sequence”，它使用 TextToInteSequence 将单词转换为整数并使用 Embedding 嵌入整数序列，或者您可以使用“ngram”，它使用 TextToNgramVector 对句子进行向量化。 您也可以不指定这些参数，这将使不同的选择自动调整。 有关详细信息，请参阅以下示例。

In [29]:
input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 14s]
val_loss: 0.39885959029197693

Best val_loss So Far: 0.39885959029197693
Total elapsed time: 00h 00m 14s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 5s 30ms/step - loss: 0.5026 - accuracy: 0.7582
Epoch 2/2
157/157 [==============================] - 5s 29ms/step - loss: 0.1930 - accuracy: 0.9394
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


AutoModel 的用法类似于 Keras 的函数式 API。 基本上，您正在构建一个图，其边是块，节点是块的中间输出。 使用 `output_node = ak.[some_block]([block_args])(input_node)` 添加从 `input_node` 到 `output_node` 的边。

您甚至还可以使用更细粒度的块来进一步自定义搜索空间。 请参阅以下示例。

In [30]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
# Use separable Conv layers in Keras.
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 15s]
val_loss: 0.6931501030921936

Best val_loss So Far: 0.6931501030921936
Total elapsed time: 00h 00m 15s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 8s 43ms/step - loss: 0.6932 - accuracy: 0.4876
Epoch 2/2
157/157 [==============================] - 6s 38ms/step - loss: 0.6932 - accuracy: 0.4944
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


## 数据格式

AutoKeras TextClassifier 对于数据格式非常灵活。

对于文本，输入数据应该是一维的。对于分类标签，AutoKeras 接受普通标签，即字符串或整数，以及单热编码的编码标签，即 0 和 1 的向量。

我们还支持对训练数据使用 tf.data.Dataset 格式。

In [31]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,))).batch(32)

clf = ak.TextClassifier(overwrite=True, max_trials=2)

In [32]:
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=2)

Trial 2 Complete [00h 00m 21s]
val_loss: 0.4119846224784851

Best val_loss So Far: 0.4119846224784851
Total elapsed time: 00h 00m 39s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
157/157 [==============================] - 11s 58ms/step - loss: 0.6733 - accuracy: 0.5650
Epoch 2/2
157/157 [==============================] - 9s 58ms/step - loss: 0.4243 - accuracy: 0.8212


/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


In [ ]:
# Predict with the best model.
predicted_y = clf.predict(test_set)

In [ ]:
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))